In [1]:
import pandas as pd
import numpy as np

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
idt = pd.read_json('https://raw.githubusercontent.com/Gapminder/waffle-server-importers-exporters/world-legacy-with-data/data/out/gw/meta/quantities.json?token=AAVecNfSWC2bIGcLqoUZKjOVZIPQpm6Kks5W66t2wA%3D%3D')

In [10]:
import re


def to_concept_id(s, sub='[/ -\.\*";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    s1 = re.sub(sub, sep, s.strip())
    s1 = re.sub(r'\[.*\]', '', s1)
    s1 = s1.replace('\n', '')

    if s1[-1] == sep:  # remove the last underscore
        s1 = s1[:-1]

    return s1.lower()


In [7]:
idt.to_csv('ddf--concepts.csv', index=False)

In [22]:
concepts = pd.read_csv('./source/graph_settings - Indicators.csv')

In [23]:
dsc = concepts.columns

In [24]:
concepts.columns = list(map(to_concept_id, concepts.columns))

In [25]:
concepts['concept_type'] = 'measure'

In [26]:
concepts = concepts.drop('download', axis=1)

In [27]:
concepts = concepts.iloc[:, [5, 0, 1,2,3,4,6,7]]

In [30]:
concepts = concepts.rename(columns={'ddf_id':'concept'})

In [31]:
concepts.head()

,concept_id,name,tooltip,menu_level1,menu_level_2,indicator_url,scale,concept_type
0,children_per_woman_total_fertility,Children per woman (total fertility),Total fertility rate. The number of children t...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure
1,co2_emissions_tonnes_per_person,CO2 emissions (tonnes per person),Carbon dioxide emissions from the burning of f...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure
2,income_per_person_gdppercapita_ppp_inflation_a...,"Income per person (GDP/capita, PPP$ inflation-...",Gross domestic product per person adjusted for...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,log,measure
3,child_mortality_0_5_year_olds_dying_per_1000_born,"Child mortality (0-5 year-olds dying per 1,000...",Death of children under five years of age per ...,NaN,NaN,http://spreadsheets.google.com/pub?key=0ArfEDs...,log,measure
4,life_expectancy_years,Life expectancy (years),The average number of years a newborn child wo...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure


In [34]:
dsc.drop('ddf_id')

Index(['Name', 'Tooltip', 'Menu level1', 'Menu level 2', 'Indicator url',
       'Download', 'Scale'],
      dtype='object')

In [35]:
len(concepts)

544

In [59]:
# todo: discrete concepts

In [36]:
con2 = pd.DataFrame([], columns=concepts.columns)

In [37]:
con2['name'] = dsc.drop('ddf_id')

In [39]:
con2['concept_id'] = con2['name'].apply(to_concept_id)

In [40]:
con2['concept_type'] = 'string'

In [55]:
gps = pd.read_json('../../ddf--gapminder_world/waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/meta/area_categorizarion.json')

In [58]:
gps

,a,dataCollectionUrl,groupings,id,n,providerUrl,sourceName
0,Regions of the world,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i265': ['i5', 'i13', 'i31', 'i35', 'i44', 'i...",CATID0,Geographic regions,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
1,How rich are you?,http://spreadsheets.google.com/pub?key=phT4mwj...,"{'i268': ['i6', 'i9', 'i12', 'i13', 'i14', 'i1...",CATID1,Income groups,http://spreadsheets.google.com/pub?key=phT4mwj...,http://spreadsheets.google.com/pub?key=phT4mwj...
2,Does the country have a coast?,http://spreadsheets.google.com/pub?key=rBwKBod...,"{'i270': ['i2', 'i3', 'i4', 'i5', 'i7', 'i8', ...",CATID2,Landlocked,http://spreadsheets.google.com/pub?key=rBwKBod...,http://spreadsheets.google.com/pub?key=rBwKBod...
3,Member of the G77 (developing country) group o...,http://spreadsheets.google.com/pub?key=tVI_UCw...,"{'i273': ['i13', 'i14', 'i21', 'i37', 'i57', '...",CATID3,G77 and OECD countries,http://spreadsheets.google.com/pub?key=tVI_UCw...,http://spreadsheets.google.com/pub?key=tVI_UCw...
4,Using 4 colors to describe the geographic regi...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i277': ['i4', 'i7', 'i23', 'i28', 'i33', 'i3...",CATID4,Geographic regions in 4 colors,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...
5,The religion of more than 50% of the religious...,http://spreadsheets.google.com/spreadsheet/pub...,"{'i279': ['i5', 'i6', 'i7', 'i8', 'i9', 'i10',...",CATID5,Main religion 2008,http://spreadsheets.google.com/spreadsheet/pub...,http://spreadsheets.google.com/spreadsheet/pub...


In [57]:
gps['groupings'][1]

{'i266': ['i1',
  'i18',
  'i23',
  'i33',
  'i34',
  'i35',
  'i40',
  'i41',
  'i48',
  'i49',
  'i68',
  'i71',
  'i80',
  'i92',
  'i93',
  'i95',
  'i114',
  'i116',
  'i120',
  'i125',
  'i132',
  'i133',
  'i136',
  'i151',
  'i152',
  'i155',
  'i162',
  'i185',
  'i202',
  'i207',
  'i222',
  'i223',
  'i226',
  'i236',
  'i257',
  'i211'],
 'i267': ['i3',
  'i4',
  'i5',
  'i7',
  'i10',
  'i15',
  'i20',
  'i22',
  'i27',
  'i28',
  'i29',
  'i32',
  'i44',
  'i47',
  'i52',
  'i55',
  'i61',
  'i62',
  'i64',
  'i74',
  'i79',
  'i87',
  'i99',
  'i103',
  'i104',
  'i109',
  'i112',
  'i113',
  'i123',
  'i126',
  'i131',
  'i134',
  'i135',
  'i138',
  'i141',
  'i143',
  'i148',
  'i153',
  'i171',
  'i172',
  'i175',
  'i183',
  'i189',
  'i191',
  'i198',
  'i201',
  'i209',
  'i215',
  'i224',
  'i228',
  'i231',
  'i232',
  'i233',
  'i235',
  'i246'],
 'i268': ['i6',
  'i9',
  'i12',
  'i13',
  'i14',
  'i16',
  'i17',
  'i19',
  'i21',
  'i24',
  'i31',
  'i37',
  

In [60]:
geo = pd.read_excel('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/country_synonyms.xlsx')

In [64]:
geo.head() 

,Use Name,Gapminder list,Alternative 1,Alternative 2,Alternative 3,Alternative 4 (CDIAC),pandg,GOD_ID,ISO3dig_ext,ALT 5,NAME,CODE,Number,Gwid,arb1,arb2,arb3,arb4,arb5,arb6
0,Abkhazia,Abkhazia,NaN,NaN,NaN,NaN,NaN,GE-AB,_abk,NaN,NaN,NaN,NaN,i0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Afghanistan,Islamic Republic of Afghanistan,NaN,NaN,Afghanistan,AFGHANISTAN,AF,afg,NaN,AFGHANISTAN,AFG,4,i1,NaN,NaN,NaN,NaN,NaN,NaN
2,Akrotiri and Dhekelia,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,Akrotiri_Dhekelia,_akr,NaN,NaN,NaN,NaN,i2,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,Albania,NaN,NaN,NaN,Albania,ALBANIA,AL,alb,NaN,ALBANIA,ALB,8,i3,NaN,NaN,NaN,NaN,NaN,NaN
4,Algeria,Algeria,NaN,NaN,NaN,Algeria,ALGERIA,DZ,dza,NaN,ALGERIA,DZA,12,i4,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
geo_ = geo[['ISO3dig_ext', 'Gwid']]

In [66]:
geo_ = geo_.set_index('Gwid')

In [70]:
geo_.get_value('i0', 'ISO3dig_ext')

'_abk'

In [110]:
def rename_col(s):
    
    real = idt[idt['-t-ind'] == s]['-t-name'].iloc[0]
    cid = concepts[concepts['name'] == real]['concept_id'].iloc[0]
#     print(real, ': ', cid)
    
    return cid

In [92]:
def rename_geo(s):
    return geo_.get_value(s, 'ISO3dig_ext')

In [71]:
import os

In [73]:
path = '../waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/indicators/'

In [74]:
fs = os.listdir(path)

In [95]:
p1 = os.path.join(path, fs[300])
d1 = pd.read_json(p1)

In [96]:
d1.geo = d1.geo.apply(rename_geo)

In [97]:
d1

,geo,rCyfwvThkbHVlNVw48vHybg,time
0,afg,69.5,1991
1,afg,69.3,1992
2,afg,69.7,1993
3,afg,69.3,1994
4,afg,69.3,1995
5,afg,69.4,1996
6,afg,69.5,1997
7,afg,69.7,1998
8,afg,69.6,1999
9,afg,69.6,2000


In [85]:
idt[idt['-t-ind'] == '0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc']['-t-name'].iloc[0]

'Murdered children 0-14 (per 100,000 people)'

In [115]:
rename_col('rCyfwvThkbHVlNVw48vHybg')

'males_aged_15_24_employment_rate_percent'

In [ ]:
# now try to make data points

In [114]:
for f in fs[1:]:
    p = os.path.join(path, f)
    
    col = f[:-5]
    col_r = rename_col(col)
  
    d = pd.read_json(p)

    if 'geo' in d.columns:
        d['geo'] = d['geo'].apply(rename_geo)
        d = d.rename(columns={col:col_r}) #ok?
        d.to_csv('./output/datapoints/ddf--datapoints--'+col_r+'--by--geo--time.csv', index=False)
    else:
        print(col, ':', col_r)
    
    

rj8WsGGNXkijP447qnY5RLg : ———————————————————————
rMF2H57_Vd8TdobvtNhYuwg : ———————————————————————
tAwlNo30o6vVgDUk0EmDolQ : ———————————————————————
tDkfgIt5oQ_ZJQ7IBmUvsUg : ———————————————————————
tdKPTXS1U5Jn3qAQICk4-Vw : ———————————————————————
teiT4rPfnWcd5mCldIm4Zhw : ———————————————————————
tFc2tyrcWLcUyGhq6mTAbDg : ———————————————————————
tFrUv6vnhhO4Jr1czdG_p0Q : ———————————————————————
tIL264Nlw0AlIWlXuCWWNmQ : ———————————————————————
tJwP0x54AZ-X-bGL-RebrpA : ———————————————————————
tLiJLI_rimqrdQRWdamPPRQ : ———————————————————————
tnDimTzGwU_BbmtOMHOUFFQ : ———————————————————————
ts7TQEIg0amXovCDzQxV5EA : ———————————————————————
tYvQLeBTuoKsyofvwGqS-9w : ———————————————————————


In [102]:
fs[1][:-5]

'0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc'

In [106]:
len(idt)

542

In [107]:
len(concepts)

544